In [1]:
import torch
from torch import nn
from torch.nn.utils import rnn
import torch.nn.functional as F
import pandas as pd
import copy
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
# from torchsummary import summary
from torchinfo import summary
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vectors,GloVe
from collections import Counter


In [7]:
vocab_size = 30000
glove = GloVe(name='6B', dim=300)

.vector_cache/glove.6B.zip: 862MB [02:44, 5.24MB/s]                               
100%|█████████▉| 399999/400000 [01:08<00:00, 5829.78it/s]


In [8]:
print(type(glove))

<class 'torchtext.vocab.vectors.GloVe'>


In [9]:
glove['hello']

tensor([-3.3712e-01, -2.1691e-01, -6.6365e-03, -4.1625e-01, -1.2555e+00,
        -2.8466e-02, -7.2195e-01, -5.2887e-01,  7.2085e-03,  3.1997e-01,
         2.9425e-02, -1.3236e-02,  4.3511e-01,  2.5716e-01,  3.8995e-01,
        -1.1968e-01,  1.5035e-01,  4.4762e-01,  2.8407e-01,  4.9339e-01,
         6.2826e-01,  2.2888e-01, -4.0385e-01,  2.7364e-02,  7.3679e-03,
         1.3995e-01,  2.3346e-01,  6.8122e-02,  4.8422e-01, -1.9578e-02,
        -5.4751e-01, -5.4983e-01, -3.4091e-02,  8.0017e-03, -4.3065e-01,
        -1.8969e-02, -8.5670e-02, -8.1123e-01, -2.1080e-01,  3.7784e-01,
        -3.5046e-01,  1.3684e-01, -5.5661e-01,  1.6835e-01, -2.2952e-01,
        -1.6184e-01,  6.7345e-01, -4.6597e-01, -3.1834e-02, -2.6037e-01,
        -1.7797e-01,  1.9436e-02,  1.0727e-01,  6.6534e-01, -3.4836e-01,
         4.7833e-02,  1.6440e-01,  1.4088e-01,  1.9204e-01, -3.5009e-01,
         2.6236e-01,  1.7626e-01, -3.1367e-01,  1.1709e-01,  2.0378e-01,
         6.1775e-01,  4.9075e-01, -7.5210e-02, -1.1

In [4]:
tokenizer = get_tokenizer('basic_english')

In [5]:
print(glove.vectors.shape)
print(type(glove.vectors))

NameError: name 'glove' is not defined

In [6]:
df = pd.read_csv('Data/train.csv')
sentences = df['text'].values
labels = df['label'].values

In [7]:
word_list = []
for sentence in sentences:
    for word in tokenizer(sentence):
        word_list.append(word)

print(len(word_list))
        

3345810


In [8]:
freq = Counter(word_list)
freq_ = sorted(freq,key=freq.get,reverse=True)

print(len(freq_))

56069


In [9]:
glove.stoi['this']
test = copy.deepcopy(glove.vectors[:240000])
print(test.shape)

NameError: name 'glove' is not defined

In [12]:
class BaseModel(nn.Module):
    '''
    input_size -> text vocab size
    '''
    def __init__(self, input_size, output_size, embedding_dim, hidden_dim, num_layers, batch_first):
        super(BaseModel, self).__init__()

        self.num_layers = num_layers
        self.batch_first = batch_first   
        self.hidden_dim = hidden_dim

        """
        TODO: Implement your own model. You can change the model architecture.
        """
        # self.embedding = nn.Embedding.from_pretrained(test,freeze=True)
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=batch_first)
        self.fc = nn.Linear(hidden_dim, output_size)

    # the size of x in forward is (seq_length, batch_size) if batch_first=False
    def forward(self, x, input_lengths):
        batch_size = x.size(0) if self.batch_first else x.size(1)

        #h_0: (num_layers * num_directions, batch_size, hidden_size)
        h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
        c_0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim)

        embedding = self.embedding(x)
        packed_input = pack_padded_sequence(embedding, input_lengths.tolist(), batch_first=self.batch_first)
        # output, hidden = self.rnn(packed_input, (h_0, c_0))
        packed_output, hidden = self.rnn(packed_input)
        # output, _ = pad_packed_sequence(output, batch_first=self.batch_first)

        output = self.fc(hidden[0][-1])

        # outputs, hidden = self.rnn(embedding, None)  # outputs.shape -> (sequence length, batch size, hidden size)

        # outputs = outputs[:, -1, :] if self.batch_first else outputs[-1, :, :]
        # output = self.fc(outputs)
        
        return output, hidden

In [13]:
model = BaseModel(240000, 2, 300, 64, 3, True)

In [14]:
model.embedding.weight.data.copy_(test)

NameError: name 'test' is not defined

In [15]:
torch.save(model.state_dict(),'test.pt')

In [16]:
from model import LSTM_ATTENTION

In [17]:
model = LSTM_ATTENTION(60000, 3, 300, 64, 3, True)


In [23]:
from dataset import *
from tqdm import tqdm

In [52]:
# Make Test Loader
test_dataset = TextDataset('./Data', 'test', 60000)
test_loader = make_data_loader(test_dataset, 32, True, shuffle=False)

In [53]:
for i,(text,label) in enumerate(tqdm(test_loader)):
    input_lengths = torch.tensor([len(x.nonzero()) for x in text])
    input_lengths, perm_idx = input_lengths.sort(0,descending=True)
    text = text[perm_idx]
    label = label[perm_idx]

    output, hidden, pad = model(text, input_lengths)

    print(text.shape)
    print(text[0])
    print(pad.data.shape)
    print(pad.batch_sizes)
    print(torch.sum(pad.batch_sizes))
    print(pad.batch_sizes.shape)
    print("--------------")
    print(output.shape)
    print(hidden[0].shape)
    break

  0%|          | 0/4 [00:00<?, ?it/s]

torch.Size([32, 30])
tensor([  981,    48,     7,  1077,    52,    11,   423,   147,    14,  3150,
         2622,    52,  3299,  2384,     6,   181,    46,  5786,    48,  2157,
        38821,   101, 13248,  3299,    17,  3490,   522,   644,    63,    22])
torch.Size([685, 128])
tensor([32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 30, 29, 29, 29, 27,
        26, 23, 20, 17, 14, 10,  7,  4,  1,  1,  1,  1])
tensor(685)
torch.Size([30])
--------------
torch.Size([32, 2])
torch.Size([6, 32, 64])


In [141]:
# print(hidden[0][0,0,0])
# print(hidden[0][1,0,0])
# print(hidden[0][2,0,0])
# print(hidden[0][3,0,0])
# print(hidden[0][4,0,0])
# print(hidden[0][5,0,0].item())
# print(pad.data[1][0].item())
# print(pad[1][0])
for i in range(685):
    for j in range(6):    
        if pad.data[i][64].item() == hidden[0][j,0,0].item():
            print(i,j)
            
# print(pad.data[-1][:64])
# print(hidden[0][4,0])

print(pad.data[-1][63].item())
print(hidden[0][5,0,0].item())

0 5
0.032375626266002655
0.03271143138408661


In [138]:
# concat hidden[0][-1] & hidden[0][-2]

h = torch.concat((hidden[0][-2],hidden[0][-1]),1)
print(h.shape)

torch.Size([32, 128])


In [137]:
# print(h[0])
# print(pad.data[684][64])
for i in range(128):
    for j in range(128):
        if pad.data[684][i].item() == h[0][j].item():
            print(i,j)
            break

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63


In [70]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [143]:
packed_output, packed_hidden = pad_packed_sequence(pad, batch_first=True)

In [144]:
print(packed_output.shape)

torch.Size([32, 30, 128])


In [142]:
print(packed_output[-1][0][0])
print(hidden[0][4][0][0])

tensor(0.0089, grad_fn=<SelectBackward0>)
tensor(0.0089, grad_fn=<SelectBackward0>)


In [150]:
inp = torch.zeros(packed_output.size(0),1).long()
inp += 2
print(inp)
print(inp.shape)

tensor([[2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2]])
torch.Size([32, 1])


In [155]:
embedding = nn.Embedding(60000, 300)

In [156]:
emb = embedding(inp)

In [157]:
emb.shape

torch.Size([32, 1, 300])

In [158]:
lstm = nn.LSTM(300,64,3,bidirectional=True,batch_first=True)

In [165]:
l,h = lstm(emb)

In [166]:
print(l.shape)

torch.Size([32, 1, 128])


In [167]:
l = l.view(l.shape[0],l.shape[2],-1)
a = torch.bmm(packed_output,l)

In [175]:
print(a.shape)
print(packed_output.shape)

torch.Size([32, 30, 1])
torch.Size([32, 30, 128])


In [172]:
att_dis = F.softmax(a,dim=1)

In [173]:
att_dis.shape

torch.Size([32, 30, 1])

In [174]:
att_dis[0]

tensor([[0.0335],
        [0.0335],
        [0.0336],
        [0.0336],
        [0.0335],
        [0.0335],
        [0.0335],
        [0.0334],
        [0.0333],
        [0.0332],
        [0.0332],
        [0.0332],
        [0.0331],
        [0.0331],
        [0.0331],
        [0.0331],
        [0.0332],
        [0.0332],
        [0.0332],
        [0.0333],
        [0.0333],
        [0.0335],
        [0.0335],
        [0.0334],
        [0.0335],
        [0.0336],
        [0.0335],
        [0.0333],
        [0.0331],
        [0.0331]], grad_fn=<SelectBackward0>)

In [176]:
att_val = torch.sum(att_dis * packed_output,dim=1)

In [177]:
att_val.shape

torch.Size([32, 128])

In [178]:
att_val[0]

tensor([ 0.0194, -0.0533, -0.0954, -0.0021,  0.0615,  0.0166, -0.0876, -0.0468,
         0.0302,  0.0076, -0.0161,  0.0373, -0.0561,  0.0250, -0.0242, -0.0376,
        -0.0156,  0.0522,  0.0137,  0.0154, -0.0069, -0.1210,  0.0198,  0.0825,
         0.0373, -0.0346, -0.0512, -0.1070,  0.0046,  0.0510,  0.0707, -0.1055,
        -0.0070,  0.0717,  0.0309, -0.0245,  0.0198, -0.0219, -0.0603, -0.0379,
        -0.0213,  0.0248, -0.0282, -0.0646, -0.0268,  0.0162, -0.1221, -0.0987,
         0.1353, -0.1139,  0.0329, -0.0816, -0.0916,  0.0314,  0.0149, -0.0383,
         0.0366,  0.0169, -0.0181,  0.0241,  0.0574,  0.1169,  0.0767, -0.0011,
         0.0021,  0.0433,  0.0403,  0.0765, -0.1053, -0.0378,  0.0569,  0.0192,
         0.0573, -0.0308,  0.0897,  0.0808,  0.0734,  0.0504, -0.0058, -0.0012,
        -0.0667, -0.0833,  0.0385, -0.0872,  0.0181, -0.0853,  0.0447,  0.0739,
         0.0485,  0.0676, -0.0354,  0.0121,  0.0255,  0.0235,  0.0017, -0.0419,
         0.0870, -0.0318, -0.1115,  0.02

In [179]:
s = att_dis * packed_output

In [180]:
s.shape

torch.Size([32, 30, 128])

In [181]:
torch.sum(att_dis,dim=1)

tensor([[1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000]], grad_fn=<SumBackward1>)

In [188]:
con = torch.cat((att_val,l.squeeze(2)),dim=1)

In [189]:
print(con.shape)

torch.Size([32, 256])


In [187]:
print(l.squeeze(2).shape)

torch.Size([32, 128])
